In [10]:
import pandas as pd
import os
from alpaca.data.historical import StockHistoricalDataClient

In [11]:
api_key = os.environ["ALPACA_KEY"]
api_secret = os.environ["ALPACA_SECRET"]
client = StockHistoricalDataClient(api_key, api_secret)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_api_key',
 '_api_version',
 '_base_url',
 '_data_get',
 '_get_auth_headers',
 '_get_default_headers',
 '_oauth_token',
 '_one_request',
 '_request',
 '_retry',
 '_retry_codes',
 '_retry_wait',
 '_return_paginated_result',
 '_sandbox',
 '_secret_key',
 '_session',
 '_use_basic_auth',
 '_use_raw_data',
 '_validate_credentials',
 '_validate_pagination',
 'delete',
 'get',
 'get_stock_bars',
 'get_stock_latest_bar',
 'get_stock_latest_quote',
 'get_stock_latest_trade',
 'get_stock_quotes',
 'get_stock_snapshot',
 'get_stock_trades',
 'patch',
 'post',
 'pu